# Project 3: Airbnb
**This is the third of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-12-05, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python/PySpark**
- **For each question you may use as many cells for your solution as you like**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**

# Introduction
[Airbnb](http://airbnb.com) is an online marketplace for arranging or offering lodgings. In this project you will use Spark to analyze data obtained from the Airbnb website. The purpose of the analysis is to extract information about trends and patterns from the data.

The project has two parts.

### Part 1: Loading, describing and preparing the data
There's quite a lot of data. Make sure that you can load and correctly parse the data, and that you understand what the dataset contains. You should also prepare the data for the analysis in part two. This means cleaning it and staging it so that subsequent queries are fast.

### Par 2: Analysis
In this part your goal is to learn about trends and usage patterns from the data. You should give solutions to the tasks defined in this notebook, and you should use Spark to do the data processing. You may use other libraries like for instance Pandas and matplotlib for visualisation.

## Guidelines
- Processing data should be done using Spark. Once data has been reduced to aggregate form, you may use collect to extract it into Python for visualisation.
- Your solutions will be evaluated by correctness, code quality and interpretability of the output. This means that you have to write clean and efficient Spark code that will generate sensible execution plans, and that the tables and visualisations that you produce are meaningful and easy to read.
- You may add more cells for your solutions, but you should not modify the notebook otherwise.

### Create Spark session and define imports

In [2]:
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.appName("SparkIntro").getOrCreate()

# Part 1: Loading, describing and preparing the data
The data comes in two files. Start by downloading the files and putting them in your `data/` folder.

- [Listings](https://files.dtu.dk/u/siPzAasj8w2gI_ME/listings.csv?l) (5 GB)
- [Reviews](https://files.dtu.dk/u/k3oaPYp6GjKBeho4/reviews.csv?l) (9.5 GB)

### Load the data
The data has multiline rows (rows that span multiple lines in the file). To correctly parse these you should use the `multiline` option and set the `escape` character to be `"`.

In [3]:
import itertools
import csv
import pandas as pd
import pyspark.sql.functions as f
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import substring, length, col, expr

##### Slicing the data

In [ ]:


entries = []
with open('/home/jovyan/work/Project3/listings.csv', 'r') as f:
    mycsv = csv.reader(f)
    for row in itertools.islice(mycsv, 10000):
        entries.append(row)

columns_ = entries[0]
df = pd.DataFrame(entries,columns=columns_)
df = df.iloc[1:-1,:]
df.head()
#df.to_csv("listings_sub_10000.csv")

##### Reading into spark

In [ ]:
df = spark.read.option("delimiter",",").option("multiline",True).option("header", True).option("escape",'"').option("inferSchema",True).csv('listings_sub_10000.csv')


In [ ]:
df.show()

In [ ]:
df.select("listing_url").show()

### Describe the data
List the features (schema) and sizes of the datasets.

In [ ]:
df.printSchema()

In [ ]:
print("rows: ", df.count(), "columns: ", len(df.columns))

In [ ]:
#Removing all rows which has null
df.where(reduce(lambda x, y: x & y,  (f.col(x).isNotNull() for x in df.columns))).count()

In [ ]:
#Removing all rows which has null
df.dropna(how='any').count()

### Prepare the data for analysis
You should prepare two dataframes to be used in the analysis part of the project. You should not be concerned with cleaning the data. There's a lot of it, so it will be sufficient to drop rows that have bad values. You may want to go back and refine this step at a later point when doing the analysis.

You may also want to consider if you can stage your data so that subsequent processing is more efficient (this is not strictly necessary for Spark to run, but you may be able to decrease the time you sit around waiting for Spark to finish things)

# Part 2: Analysis
Use Spark and your favorite tool for data visualization to solve the following tasks.

## The basics
Compute and show a dataframe with the number of listings and neighbourhoods per city.

Based on the table above, you should choose a city that you want to continue your analysis for. The city should have mulitple neighbourhoods with listings in them.

Compute and visualize the number of listings of different property types per neighbourhood in your city.

## Prices
Compute the minimum, maximum and average listing price in your city. 

In [ ]:
city = "Stockholm"

In [ ]:
df_s = df.filter(f.col("city")==city)

In [ ]:
df_s.count()

In [ ]:
#df_s.price = df_s.price.cast('float')

In [ ]:
from pyspark.sql.types import *

In [ ]:
from pyspark.sql.functions import substring, length, col, expr
df_s = df_s.withColumn("price",expr("substring(price, 2, length(price))"))

In [ ]:
from  pyspark.sql.functions import regexp_replace,col

In [ ]:
df_s = df_s.withColumn("price", regexp_replace(f.col("price"), ",", ""))

In [ ]:
#df_s.show()

In [ ]:
#df_s.select(df_s.price.cast("float").alias('price_new')).collect()

In [ ]:
from pyspark.sql.types import *
df_d = df_s.withColumn("price",df_s["price"].cast(DoubleType()))

In [ ]:
#df_d.show()

In [ ]:
print("Basic statistics for", city)
df_d.select(f.avg('price').alias("Averge price"), f.max('price').alias("Max price"), f.min('price').alias("Min price")).show()

Compute and visualize the distribution of listing prices in your city.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
sns.set(rc={'figure.figsize':(16,6)})

In [ ]:
df_city_p = df_d.toPandas()

In [ ]:
sns.distplot(df_city_p.price, kde=False, rug=False, bins=150);

The value of a listing is its rating divided by its price.

Compute and show a dataframe with the 3 highest valued listings in each neighbourhood.

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
df_d = df_d.withColumn("review_scores_rating",df_s["review_scores_rating"].cast(DoubleType()))

In [ ]:
df_d = df_d.withColumn("value_listing",lit(f.col("review_scores_rating")/f.col("price")))

In [ ]:
df_value = df_d.select(f.col("city"),f.col("neighbourhood"),(f.col("value_listing")))

In [ ]:
#df_value.orderBy(df_value.value_listing.desc()).show()

In [ ]:
sorted_by_value = Window.partitionBy('neighbourhood').orderBy(f.desc('value_listing'))

In [ ]:
ranked_by_value = df_d.withColumn('value_listing_rank', f.rank().over(sorted_by_value))

In [ ]:
ranked_df = ranked_by_value.filter(f.col('value_listing_rank') <= 3).drop('value_listing_rank').orderBy('neighbourhood', f.desc('value_listing'))

In [ ]:
ranked_df.select(f.col("neighbourhood"),f.col("value_listing"),f.col("city"),f.col("name")).show(50)

## Trends
Now we want to analyze the "popularity" of your city. The data does not contain the number of bookings per listing, but we have a large number of reviews, and we will assume that this is a good indicator of activity on listings.

Compute and visualize the popularity (i.e., number of reviews) of your city over time.

Compute and visualize the popularity of neighbourhoods over time. If there are many neighbourhoods in your city, you should select a few interesting ones for comparison.

Compute and visualize the popularity of your city by season. For example, visualize the popularity of your city per month.

## Reviews
In this part you should determine which words used in reviews that are the most positive. 

The individual reviews do not have a rating of the listing, so we will assume that each review gave the average rating to the listing, i.e., the one on the listing.

You should assign a positivity weight to each word seen in reviews and list the words with the highest weight. It is up to you to decide what the weight should be. For example, it can be a function of the rating on the listing on which it occurs, the number of reviews it occurs in, and the number of unique listings for which it was used to review.

Depending on your choice of weight function, you may also want to do some filtering of words. For example, remove words that only occur in a few reviews.

In [ ]:
import itertools
import csv

entries = []
with open('../Project3/reviews.csv', 'r') as f:
    mycsv = csv.reader(f)
    for row in itertools.islice(mycsv, 10000):
        entries.append(row)
        
import pandas as pd
columns_ = entries[0]
df = pd.DataFrame(entries,columns=columns_)
df = df.iloc[1:-1,:]
#df.head()
#df.to_csv("reviews_sub_10000.csv")

In [ ]:
r = pd.read_csv("reviews_sub_10000.csv")
l = pd.read_csv("listings_sub_10000.csv")

In [ ]:
r.head()

In [ ]:
l.head()

From the listings we need to:
- calculate the average rating for each ID
- join the average rating on to the ratings

In [26]:
dfl = spark.read.option("delimiter",",").option("multiline",True).option("header", True).option("escape",'"').option("inferSchema",True).csv('listings_sub_10000.csv')

In [27]:
dfr = spark.read.option("delimiter",",").option("multiline",True).option("header", True).option("escape",'"').option("inferSchema",True).csv('reviews_sub_10000.csv')

In [28]:
mean_scores = dfl.groupBy("id").agg(f.mean("review_scores_rating").alias("average_rating"))
mean_scores = mean_scores.select(f.col("id").alias("id_"), f.col("average_rating"))

In [29]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [30]:
mean_scores.show()

+--------+--------------+
|     id_|average_rating|
+--------+--------------+
|  438129|          95.0|
| 2466500|          99.0|
| 3361594|         100.0|
| 4045800|         100.0|
| 4892006|          95.0|
| 5436847|          90.0|
| 6084492|         100.0|
| 6765153|          93.0|
| 7046029|         100.0|
| 7746807|         100.0|
| 8374408|          95.0|
|11245734|         100.0|
|12418867|         100.0|
|13048954|          99.0|
|13639296|          98.0|
|16322150|          95.0|
|17601470|          87.0|
|17603444|          90.0|
|17732905|          98.0|
|17742573|          97.0|
+--------+--------------+
only showing top 20 rows



In [31]:
dfr = dfr.join(mean_scores, (f.col('listing_id') == f.col('id_')), 'inner').drop('id_')

In [32]:
dfr.show()

+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------+
| _c0|listing_id|      id|               date|reviewer_id|     reviewer_name|            comments|average_rating|
+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------+
|9766|   1271843| 8513212|2013-11-03 00:00:00|    4641577|            Pamela|André was my firs...|          99.0|
|9767|   1271843| 9043505|2013-12-02 00:00:00|    6890450|              Emma|Andre was the bes...|          99.0|
|9768|   1271843| 9533868|2014-01-01 00:00:00|    9496791|   Julie And Chris|André was a fanta...|          99.0|
|9769|   1271843|10049736|2014-01-27 00:00:00|     940929|             Denis|It was a right ch...|          99.0|
|9770|   1271843|10324840|2014-02-13 00:00:00|   11454004|             Filip|We stayed in Stoc...|          99.0|
|9771|   1271843|10396795|2014-02-17 00:00:00|   10505993|             Pablo|Stop lookin

A simple way to assess how positive each word is:
- for every review: tokenize sentence
- create a bag of words --> a document-term matrix
- create a dictionary, containing every word from the reviews and create an aggregated score on each word. The aggreated word review could be as simple as:
    - adding all the average reviews and dividing with the total number of reviews it has been a part of (to account for non-frequent words) 
    - creating a much simpler score, where bad reviews= -1, middle= 0, high= 1. These simple metrics can then be added up and normalised again. 

In [ ]:
dfl.groupBy('country').agg(f.sum('review_scores_rating')).filter(f.col('country') == 'Sweden').explain()

In [ ]:
dfl.filter(f.col('country') == 'Sweden').groupBy('country').agg(f.sum('review_scores_rating')).explain()